<a href="https://colab.research.google.com/github/kstyle2198/NLP_TIPS/blob/main/Few_Shot_Text_Classification(%EB%B6%88%EA%B7%A0%ED%98%95%EB%8D%B0%EC%9D%B4%ED%84%B0).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://medium.com/@knowledgrator/achieve-90-results-in-few-shot-text-classification-with-just-0-1-data-6bebdec1e08f

In [1]:
!pip install datasets transformers accelerate setfit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.2/74.2 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 9.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 15.7 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=296155319b20e55f233f3d4b3c5faaf0ba62499fbf5466edfe8c35b7f881cf72
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built se

# Load Dataset

In [3]:
from datasets import load_dataset

#emotion
emotion_dataset = load_dataset("dair-ai/emotion")

test_dataset = emotion_dataset['test']

classes = test_dataset.features["label"].names

N = 8
train_dataset = emotion_dataset['train'].shuffle(seed=41).select(range(len(classes)*N))

/usr/local/lib/python3.10/dist-packages/datasets/load.py:1429: FutureWarning: The repository for dair-ai/emotion contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/dair-ai/emotion
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split:   0%|          | 0/16000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [6]:
emotion_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})

In [4]:
classes

['sadness', 'joy', 'love', 'anger', 'fear', 'surprise']

In [5]:
train_dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 48
})

# SetFit

Firstly, we will see what results we can achieve with SetFit — an alternative few-shot learning approach that uses text embeddings for classification. Then, we will see how our approach is more powerful than SetFit.

In [33]:
from setfit import SetFitModel, Trainer, TrainingArguments
from sklearn.metrics import classification_report

In [34]:
model = SetFitModel.from_pretrained("BAAI/bge-base-en-v1.5")


args = TrainingArguments(
   batch_size=32,
   num_epochs=3,
)


trainer = Trainer(
   model=model,
   args=args,
   train_dataset=train_dataset,
   eval_dataset=test_dataset,
)

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


In [35]:
trainer.train()

***** Running training *****
  Num examples = 53
  Num epochs = 3
  Total optimization steps = 159
  Total train batch size = 32


Step,Training Loss


In [36]:
preds = model.predict(test_dataset['text'])
preds

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


tensor([0, 0, 0,  ..., 1, 1, 0])

In [37]:
print(classification_report(test_dataset['label'], preds, target_names=classes, digits=4))

              precision    recall  f1-score   support

     sadness     0.5059    0.8124    0.6235       581
         joy     0.6848    0.8345    0.7523       695
        love     0.1429    0.1635    0.1525       159
       anger     0.2632    0.0364    0.0639       275
        fear     0.0000    0.0000    0.0000       224
    surprise     0.0000    0.0000    0.0000        66

    accuracy                         0.5440      2000
   macro avg     0.2661    0.3078    0.2654      2000
weighted avg     0.4325    0.5440    0.4635      2000



/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python

# Comprehend-it method

In [13]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_name = 'knowledgator/comprehend_it-base'

tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForSequenceClassification.from_pretrained(model_name)
model

tokenizer_config.json:   0%|          | 0.00/384 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/8.66M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.06k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/738M [00:00<?, ?B/s]

DebertaV2ForSequenceClassification(
  (deberta): DebertaV2Model(
    (embeddings): DebertaV2Embeddings(
      (word_embeddings): Embedding(128100, 768, padding_idx=0)
      (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine=True)
      (dropout): StableDropout()
    )
    (encoder): DebertaV2Encoder(
      (layer): ModuleList(
        (0-11): 12 x DebertaV2Layer(
          (attention): DebertaV2Attention(
            (self): DisentangledSelfAttention(
              (query_proj): Linear(in_features=768, out_features=768, bias=True)
              (key_proj): Linear(in_features=768, out_features=768, bias=True)
              (value_proj): Linear(in_features=768, out_features=768, bias=True)
              (pos_dropout): StableDropout()
              (dropout): StableDropout()
            )
            (output): DebertaV2SelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine

In [14]:
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorWithPadding
from datasets import Dataset
import random
import torch
import evaluate
import numpy as np


In [16]:
accuracy = evaluate.load("accuracy")
accuracy

EvaluationModule(name: "accuracy", module_type: "metric", features: {'predictions': Value(dtype='int32', id=None), 'references': Value(dtype='int32', id=None)}, usage: """
Args:
    predictions (`list` of `int`): Predicted labels.
    references (`list` of `int`): Ground truth labels.
    normalize (`boolean`): If set to False, returns the number of correctly classified samples. Otherwise, returns the fraction of correctly classified samples. Defaults to True.
    sample_weight (`list` of `float`): Sample weights Defaults to None.

Returns:
    accuracy (`float` or `int`): Accuracy score. Minimum possible value is 0. Maximum possible value is 1.0, or the number of examples input, if `normalize` is set to `True`.. A higher score means higher accuracy.

Examples:

    Example 1-A simple example
        >>> accuracy_metric = evaluate.load("accuracy")
        >>> results = accuracy_metric.compute(references=[0, 1, 2, 0, 1, 2], predictions=[0, 1, 1, 2, 1, 0])
        >>> print(results)
    

In [17]:
def transform_dataset(dataset, classes, template = '{}'):
   new_dataset = {'sources':[], 'targets': [], 'labels': []}

   texts = dataset['text']
   labels = dataset['label']

   label2count = {}
   for label in labels:
       if label not in label2count:
           label2count[label]=1
       else:
           label2count[label]+=1

   count = len(labels)

   label2prob = {label:lc/count for label, lc in label2count.items()}
   unique_labels = list(label2prob)
   probs = list(label2prob.values())

   ids = list(range(len(labels)))
   for text, label_id in zip(texts, labels):
       label = classes[label_id]
       for i in range(len(classes)-1):
           new_dataset['sources'].append(text)
           new_dataset['targets'].append(template.format(label))
           new_dataset['labels'].append(1.)

       for i in range(len(classes)-1):
           neg_class_ = label
           while neg_class_==label:
               # neg_class_ = random.sample(classes, k=1)[0]
               neg_lbl = np.random.choice(unique_labels, p=probs)
               neg_class_ = classes[neg_lbl]

           new_dataset['sources'].append(text)
           new_dataset['targets'].append(template.format(neg_class_))
           new_dataset['labels'].append(-1.)
   return Dataset.from_dict(new_dataset)

In [18]:
def compute_metrics(eval_pred):
   predictions, labels = eval_pred

   predictions = np.argmax(predictions, axis=1)

   return accuracy.compute(predictions=predictions, references=labels)

In [19]:
def tokenize_and_align_label(example):
   hypothesis = example['targets']

   seq = example["sources"]+hypothesis

   tokenized_input = tokenizer(seq, truncation=True, max_length=512,
                                                    padding="max_length")

   label = example['labels']
   if label==1.0:
       label = torch.tensor(1)
   elif label==0.0:
       label = torch.tensor(2)
   else:
       label = torch.tensor(0)
   tokenized_input['label'] = label
   return tokenized_input

In [38]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
data_collator

DataCollatorWithPadding(tokenizer=DebertaV2TokenizerFast(name_or_path='knowledgator/comprehend_it-base', vocab_size=128000, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '[CLS]', 'eos_token': '[SEP]', 'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128000: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}, padding=True, max_length=None, 

In [21]:
dataset = transform_dataset(train_dataset, classes)
dataset

Dataset({
    features: ['sources', 'targets', 'labels'],
    num_rows: 480
})

In [22]:
tokenized_dataset = dataset.map(tokenize_and_align_label)
tokenized_dataset

Map:   0%|          | 0/480 [00:00<?, ? examples/s]

Dataset({
    features: ['sources', 'targets', 'labels', 'input_ids', 'token_type_ids', 'attention_mask', 'label'],
    num_rows: 480
})

In [23]:
tokenized_dataset = tokenized_dataset.train_test_split(test_size=0.1)
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['sources', 'targets', 'labels', 'input_ids', 'token_type_ids', 'attention_mask', 'label'],
        num_rows: 432
    })
    test: Dataset({
        features: ['sources', 'targets', 'labels', 'input_ids', 'token_type_ids', 'attention_mask', 'label'],
        num_rows: 48
    })
})

In [24]:
training_args = TrainingArguments(
   output_dir='comprehendo',
   learning_rate=3e-5,
   per_device_train_batch_size=8,
   per_device_eval_batch_size=8,
   num_train_epochs=3,
   weight_decay=0.01,
   evaluation_strategy="epoch",
)

In [25]:
trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_dataset["train"],
   eval_dataset=tokenized_dataset['test'],
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics,
)

In [26]:
trainer.train()

You're using a DebertaV2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.484008,0.812500
2,No log,0.309051,0.937500
3,No log,0.263137,0.958333


TrainOutput(global_step=162, training_loss=0.28751505157093943, metrics={'train_runtime': 63.1142, 'train_samples_per_second': 20.534, 'train_steps_per_second': 2.567, 'total_flos': 341001104670720.0, 'train_loss': 0.28751505157093943, 'epoch': 3.0})

In [27]:
trainer.save_model('comprehender')

In [29]:
from transformers import pipeline
from sklearn.metrics import classification_report
from tqdm import tqdm

classifier = pipeline("zero-shot-classification", model='comprehender',tokenizer=tokenizer, device=device)

In [30]:
preds = []
label2idx = {label: id for id, label in enumerate(classes)}

for example in tqdm(test_dataset):
   pred = classifier(example['text'],classes)['labels'][0]
   idx = label2idx[pred]
   preds.append(idx)
preds[:3]

  0%|          | 10/2000 [00:03<09:22,  3.54it/s]/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
100%|██████████| 2000/2000 [04:47<00:00,  6.96it/s]


[0,
 0,
 0,
 1,
 5,
 4,
 3,
 5,
 5,
 0,
 3,
 2,
 4,
 1,
 1,
 0,
 2,
 0,
 3,
 1,
 1,
 1,
 1,
 1,
 5,
 4,
 3,
 0,
 4,
 1,
 0,
 3,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 3,
 2,
 1,
 0,
 1,
 3,
 2,
 5,
 4,
 1,
 0,
 4,
 1,
 4,
 1,
 2,
 0,
 1,
 1,
 0,
 0,
 0,
 4,
 1,
 0,
 5,
 0,
 1,
 4,
 0,
 2,
 0,
 5,
 1,
 2,
 3,
 1,
 0,
 1,
 2,
 1,
 0,
 0,
 2,
 0,
 0,
 2,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 5,
 3,
 2,
 1,
 0,
 0,
 2,
 0,
 0,
 3,
 5,
 3,
 0,
 1,
 0,
 0,
 3,
 1,
 4,
 1,
 0,
 1,
 1,
 5,
 2,
 1,
 0,
 1,
 0,
 5,
 0,
 2,
 0,
 3,
 0,
 0,
 5,
 1,
 5,
 3,
 0,
 5,
 4,
 3,
 0,
 1,
 5,
 0,
 1,
 1,
 2,
 0,
 1,
 5,
 4,
 5,
 0,
 2,
 5,
 0,
 2,
 0,
 1,
 1,
 3,
 0,
 0,
 1,
 1,
 4,
 5,
 1,
 1,
 2,
 0,
 0,
 1,
 3,
 1,
 1,
 0,
 3,
 1,
 1,
 3,
 1,
 5,
 1,
 5,
 1,
 4,
 1,
 0,
 1,
 5,
 2,
 1,
 1,
 2,
 3,
 0,
 5,
 5,
 3,
 1,
 5,
 1,
 2,
 1,
 1,
 1,
 1,
 0,
 4,
 4,
 0,
 0,
 0,
 1,
 5,
 1,
 3,
 5,
 0,
 3,
 1,
 1,
 4,
 1,
 0,
 0,
 1,
 3,
 3,
 1,
 0,
 0,
 0,
 1,
 1,
 4,
 1,
 0,
 1,
 0,
 2,
 0,
 5,
 4,
 5,
 5,
 5,
 0,
 3,
 1,
 1,


In [31]:
print(classification_report(test_dataset['label'], preds,
                                        target_names=classes, digits=4))

              precision    recall  f1-score   support

     sadness     0.6755    0.6558    0.6655       581
         joy     0.7586    0.6647    0.7086       695
        love     0.3588    0.3836    0.3708       159
       anger     0.6621    0.5273    0.5870       275
        fear     0.6258    0.4330    0.5119       224
    surprise     0.1519    0.6515    0.2464        66

    accuracy                         0.5945      2000
   macro avg     0.5388    0.5527    0.5150      2000
weighted avg     0.6545    0.5945    0.6152      2000

